### Import all Packages ###

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt
import collections
import nest_asyncio
%reload_ext tensorboard
nest_asyncio.apply()
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist

### Load in TFF - EMNIST Data Set ###

In [2]:
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data(only_digits=True)

### Create a Server Class ###

In [4]:
class server:
    
    def __init__(self, numOfClients):
        self.numOfClients = numOfClients #Num of Client
        self.clientIds = list(range(0,self.numOfClients)) #Client List
        self.updateRoundNum = 0  # Update Round Number
        self.serverModel = self.createBaseModel() #Create a Keras Model for MNIST
        self.clientActive = [] #If client is available or not based on time - day/night
        self.clientCIF = []
        self.clientSelected = []
        self.maxClientsPerRound = 10
        self.updateFromClients=[]
        self.__serverTestData_X=[]
        self.__serverTestData_Y=[]
        self.__serverTestData = self.setServerTestDataTFF(emnist_test)
        self.predictionAcc=[]
        self.predictionLoss=[]
        self.__serverTrainData = self.setServerTrainDataTFF(emnist_train)
        
    def createBaseModel(self):
        #return tf.keras.models.Sequential([
        #    Dense(64, activation='relu',input_shape=(784,)),
        #    Dense(64, activation='relu'),
        #    Dense(10, activation='softmax'),])
        return tf.keras.models.Sequential([
            tf.keras.layers.Input(shape=(784,)),
            tf.keras.layers.Dense(10, kernel_initializer='zeros'),
            tf.keras.layers.Softmax()])
    
    def setServerTestData(self,Xtest,Ytest):  # NOT USED IN THIS CASE
        self.__serverTestData_X=Xtest
        self.__serverTestData_Y=Ytest
        
    def setServerTestDataTFF(self,serverTestData):  #Creates a test set for server
        BATCH_SIZE = 1000
        SHUFFLE_BUFFER = 100
        PREFETCH_BUFFER= 10

        def preprocess(dataset):
            def batch_format_fn(element):
                """Flatten a batch `pixels` and return the features as an `OrderedDict`."""
                return collections.OrderedDict(
                    x=tf.reshape(element['pixels'], [-1, 784]),
                    y=tf.reshape(element['label'], [-1, 1]))

            return dataset.repeat(10).shuffle(SHUFFLE_BUFFER).batch(
              BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)
        
        serverData1 = serverTestData.create_tf_dataset_for_client(serverTestData.client_ids[10])
        for i in range (80,500):
            serverData1 = serverData1.concatenate(serverTestData.create_tf_dataset_for_client(serverTestData.client_ids[i]))
            
        serverData =tfds.as_numpy(preprocess(serverData1))
        return serverData
     
    def setServerTrainDataTFF(self,serverTrainData):  
        #Creates a train set for server -- used for general only server training case i.e. no client all data trained on server
        # and for pre-training case evaluation i.e pre-trained model distributed to clients instead of random initializing
        #uses the emnist train dataset instead of the test data set
        
        BATCH_SIZE = 100
        SHUFFLE_BUFFER = 100
        PREFETCH_BUFFER= 10

        def preprocess(dataset):
            def batch_format_fn(element):
                """Flatten a batch `pixels` and return the features as an `OrderedDict`."""
                return collections.OrderedDict(
                    x=tf.reshape(element['pixels'], [-1, 784]),
                    y=tf.reshape(element['label'], [-1, 1]))

            return dataset.repeat(50).shuffle(SHUFFLE_BUFFER).batch(
              BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)
        
        serverData2 = serverTrainData.create_tf_dataset_for_client(serverTrainData.client_ids[1])
        for i in range (2000,2500):
            serverData2 = serverData2.concatenate(serverTrainData.create_tf_dataset_for_client(serverTrainData.client_ids[i]))
            
        serverData =tfds.as_numpy(preprocess(serverData2))
        return serverData
    
    def initialBroadcast(self):
        for i in self.clientIds:
            clientName = client_list["client_"+str(self.clientIds[i])]  #Create Client Name, Using Client_List Dictionary
            #clientName = "client_"+str(self.clientIds[i])  #Create Client Name
            #print(clientName)
            clientName.setInitialModel(self.serverModel) #setInitialModel-> Method of client Class
            #eval(clientName).setInitialModel(self.serverModel) #setInitialModel-> Method of client Class
            
    def getClientActiveStatus(self):  #if the client is available for update or not randomly set in client
        print("--------------------------------------------\n","Round NO:",self.updateRoundNum)
        self.clientActive = []
        for i in self.clientIds:
            clientName = client_list["client_"+str(self.clientIds[i])]  #Create Client Name
            #print(clientName)
            if(clientName.sendActiveStatus() == 1): #getActiveStatus() -> method of client class
                self.clientActive.append(i)
        print("Active Clients in Round :--: ", self.clientActive)
    
    
    def getClientCIF(self): # Get Client Importance Factor for active clients
        self.clientCIF=[]
        for i in self.clientActive:
            clientName = client_list["client_"+str(i)]
            c_cif = clientName.sendCIF()   #sendCIF() - method of client
            self.clientCIF.append(c_cif)
        print("Clients with Acceptable CIF: ", self.clientCIF)
        
    def getClientSelected(self): # Select The Top (N = maxClientsPerRound) with the highest CIF value
        self.clientSelected = []
        # Use non-class, local variables to leave class variables clean
        clientCIF, clientActive = ( list(t) for t in zip(*sorted(zip(self.clientCIF, self.clientActive)))) # Sort Active Clients and their CIF values, by CIF
        self.clientSelected  = clientActive[-self.maxClientsPerRound:] # Select the N=maxClientsPerRound of clients with highest CIF
        self.clientCIF = clientCIF[-self.maxClientsPerRound:] # Select the N=maxClientsPerRound of highest CIF
            
    def getRandomClients(self):
        self.clientSelected = np.random.randint(0, 100, 10)
        self.clientCIF=[]*0
        self.clientActive=[]*0
    
    def getModelUpdateFromClients(self):
        
        print("Active Clients: ",self.clientActive)
        print("CIF of Active Clients: ", self.clientCIF)
        print("Selected Clients: ", self.clientSelected)
        #print("Training Clients with CIF > 10")
        self.updateRoundNum +=1
        self.updateFromClients=[]
        #self.dataPointsClients=[]
        
        for i in range (0,len(self.clientSelected)):
            #if self.clientCIF[i] >= 10:  #Select Client only if CIF > some value 10 chosen for testing ONLY
            clientName = client_list["client_"+str(int(self.clientSelected[i]))]
            self.updateFromClients.append(clientName.sendClientUpdate()) #Get a list of updates from selected clients

        avgModelWeights = self.computeFedAvg(self.updateFromClients)
        #print("average model weights = \n",avgModelWeights)
        self.serverModel.set_weights(avgModelWeights)   
        
    def computeFedAvg(self,updates):  #Compute Federated Averaging from Available Clients
        totalDataPoints = 0
        scaleFactor=[]
        for i in range (0,len(updates)):
            totalDataPoints += updates[i][1]   #Sum the total Data Points on All Available Clients
            scaleFactor.append(updates[i][1])  #Store individual number datapoints for clients

        scaleFactor = np.array(scaleFactor)/totalDataPoints #Create the scale factor
       
        sumOfAvgWeights = []*len(updates[0][0]) 
        
        for j in range (0,len(updates[0][0])): #range of layers
            k=np.zeros_like(updates[0][0][j])
            #print("ShapeofK:",k.shape)
            for i in range(0,len(updates)): #range of clients
                k=k+updates[i][0][j]*scaleFactor[i]
                #print("ShapeofK:",k.shape)
            sumOfAvgWeights.append(k)
      
        return sumOfAvgWeights   
    
    def testServerModel(self):
        self.serverModel.compile(loss = 'sparse_categorical_crossentropy',
                                optimizer = tf.keras.optimizers.SGD (learning_rate=.01),
                                metrics=['accuracy'])
        testData = next(iter(self.__serverTestData))
        return self.serverModel.evaluate(testData['x'],
                                         testData['y'],)
        
        
    def updateAllClients(self):
        for i in self.clientIds:
            clientName = client_list["client_"+str(self.clientIds[i])]  #Create Client Name
            clientName.setModelUpdateWeights(self.serverModel.get_weights())
    
    def getDataCost(self):
        totalDataCost=0
        for i in self.clientIds:
            clientName = client_list["client_"+str(self.clientIds[i])]
            totalDataCost += clientName.getDataCost()
        print("Total Data Cost = ",totalDataCost)
        return totalDataCost
    
    def performServerTraining(self):
        trainData = next(iter(self.__serverTrainData))
        self.serverModel.compile(loss = 'sparse_categorical_crossentropy',
                                 optimizer = tf.keras.optimizers.SGD (learning_rate=.01),
                                 metrics=['accuracy'])
        self.serverModel.fit(trainData['x'],
                                       trainData['y'],
                                       epochs=1,
                                       verbose=0,shuffle=True)


### Create a Client Class ###

In [5]:
class client:
    
    def __init__(self,ID):
        self.id = ID
        self.__clientModel = tf.keras.models.Sequential()
        self.__clientModelWeights=[]
        self.activeStatus = 0
        self.__clientDataX=[] #Private to Client
        self.__clientDataY=[]
        self.clientCIF=0
        self.meanSINR = int(np.random.normal(15, 2.5, 1)) #Select random int from normal dist 
        self.roundSINR = 0
        self.propFair = 0
        self.epochs = 10
        self.lr = 0.01
        self.__clientData = self.__clientDataFromTFF(emnist_train)
        self.__currentData = next(iter(self.__clientData))
        self.dataQuality = 0
        self.dataCost = 0
        self.dataCostFactor=0.2
        
        
    def __clientDataFromTFF(self,tffDataset):
        NUM_EPOCHS = self.epochs
        #BATCH_SIZE = 20
        BATCH_SIZE = 50
        SHUFFLE_BUFFER = 100
        PREFETCH_BUFFER= 10

        def preprocess(dataset):
            def batch_format_fn(element):
                """Flatten a batch `pixels` and return the features as an `OrderedDict`."""
                return collections.OrderedDict(
                    x=tf.reshape(element['pixels'], [-1, 784]),
                    y=tf.reshape(element['label'], [-1, 1]))

            return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER).batch(
              BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)
        
        return tfds.as_numpy(preprocess(tffDataset.create_tf_dataset_for_client(tffDataset.client_ids[self.id])))
    
    
    #Methods Private to client
    def __setActiveStatus(self): 
        self.activeStatus = int(np.random.randint(0,2,1))  #01.01.21 Random Client Activation
        #self.activeStatus = 1
    
    def __setRoundSINR(self):
        self.roundSINR = np.round((np.random.normal(self.meanSINR, 1, 1)[0]), 2)
        #print("Round SNR for Client: ",self.id," is: ", self.roundSINR)
                               
    def __setPropFair(self):
        self.propFair = self.roundSINR / self.meanSINR
        
    def __setdataQuality(self):\
        
        ## Calculate for entropy - Start ##
        
#         for values in classification_probabilities:
#           for value in values:
#             score += value * np.log(value)
#         score = score * -1

        ## Calculate for entropy - End ##
        
        #self.dataQuality = 20 # Set 20 so that CIF value will always be passing
        
        # Next Client Data to be called  <<_ NOt written yet
        self.__currentData = next(iter(self.__clientData))
        predictions = self.__clientModel.predict(self.__currentData['x'])  # Get Class Predictions using Predict
        entropy = []*len(predictions)  # Entropy Will hold the entropy scores for all samples in the given round for this round
       
        for i in range (0,len(predictions)):
            sampleClassProb = predictions[i] #Take Class probability for every sample 
            score=0
            for value in sampleClassProb:  # Calculate the score - same as what jonny wrote in the pseudo code above
                #print('Value:',value)
                score += value*np.log2(value)
            score=score*(-1)
            #print('Score:',score)
            entropy.append(score)  # Append the Entropy for the given Sample
        #print('Entropy:',entropy)
        self.dataQuality = max(entropy) #Use the max (entropy) value as the data quality index
        #self.dataQuality = sum(entropy)/len(entropy)
        #print("Data Quality for Client: ",self.id," is: ", self.dataQuality)
        #self.dataQuality = (self.dataQuality-3)*100 #<<- All entropys are 3.something so normalized to get the dec values
        #print("Data Quality for Client: ",self.id," is: ", self.dataQuality)
        
    def __setCIF(self):
        #self.clientCIF = int(np.random.randint(0,50,1))  # Random Number between 0 and 49 for check purpose only
        self.__setRoundSINR()
        self.__setPropFair() #This is currently unused
        self.__setdataQuality()
        self.clientCIF = self.dataQuality + self.roundSINR 
        #self.clientCIF = (-1/self.roundSINR) + self.dataQuality #Formula for computing client CIF
        #self.clientCIF = 20
        print("CIF for Client: ",self.id," is: ", self.clientCIF)
        
    def __getTrainData(self):
        return self.__clientDataX, self.__clientDataY  # Just for Testing will be expanded to select meaningful data
    
    #Public Methods
    def sendActiveStatus(self):
        self.__setActiveStatus()
        return self.activeStatus
    
    def sendCIF(self):
        self.__setCIF()
        return self.clientCIF
    
    def setInitialModel(self,model):
        self.__clientModel=model
        print(self.__clientModel.summary)
         
    def setClientData(self,X,Y):
        self.__clientDataX = X
        self.__clientDataY = Y
   

    def sendClientUpdate(self):
        self.dataCost += self.dataCostFactor
        print("->",self.id,end=" ")
        self.__clientModel.compile(loss = 'sparse_categorical_crossentropy',
                                optimizer = tf.keras.optimizers.SGD (learning_rate=self.lr),
                                metrics=['accuracy'])
        #print(self.__clientDataX.shape)
        #print(self.__clientDataY.shape)
        if len(self.__clientDataX) > 0:
            history = self.__clientModel.fit(self.__clientDataX,
                                   tf.keras.utils.to_categorical(self.__clientDataY),
                                   epochs=self.epochs,
                                   validation_split=0.2,
                                   verbose=0,
                                    shuffle=True)

            updatedWeights = self.__clientModel.get_weights()
        
        else:
            #currentData = next(iter(self.__clientData))  # <-- Shifted to the setDataQuality Function
            history = self.__clientModel.fit(self.__currentData['x'],
                                   self.__currentData['y'],
                                   epochs=self.epochs,
                                   verbose=0,shuffle=True)
            
            
            
            
            updatedWeights = self.__clientModel.get_weights()
       
        
        numOfDataPoints = len(self.__currentData['x'])   #<-- Set the correct type to get the number of datapoints in the current update round

        return (updatedWeights,numOfDataPoints)
    
    #def logTraningHistory(self,history,roundNumber):
    #    loss_history = history.history["loss"]
    #    acc_history = history.history["accuracy"]
    #    numpy_loss_history = numpy.array(loss_history)
    
    def setModelUpdateWeights(self,modelUpdateWeights):
        self.dataCost += self.dataCostFactor
        self.__clientModel.set_weights(modelUpdateWeights)
        #self.__clientModel.
        
    def plotClientData(self):
        #print("Y_Data",self.__clientDataY)
        id0=[]
        id1=[]
        id2=[]
        id3=[]
        id4=[]
        id5=[]
        id6=[]
        id7=[]
        id8=[]
        id9=[]
        for i in range (0,len(self.__clientData['x'])):
            for j in range (0,10):
                if j==self.__clientData['y']:
                    eval('id'+str(j)).append(i)
        xAxis=['0','1','2','3','4','5','6','7','8','9']
        yAxis=[len(id0),len(id1),len(id2),len(id3),len(id4),len(id5),len(id6),len(id7),len(id8),len(id9)]
        plt.bar(xAxis,yAxis)
        title="Data for Client"+str(self.id)
        plt.title(title)
        plt.show()
    
    def getDataCost(self):
        return self.dataCost

### Initialize Clients ###

In [6]:
client_list = {}

# Set the number of clients
num_clients = 100
for i in range(num_clients):
    name = "client_" + str(i)
    client_list[name] = client(i)

### Initialize Server ###

In [7]:
# Initialize server with number of clients
serverFA = server(num_clients)

### Initial Broadcast from Server to Client ###

In [55]:
serverFA.initialBroadcast()

<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000027CB661ADC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000027CB661ADC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000027CB661ADC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000027CB661ADC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000027CB661ADC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000027CB661ADC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000027CB661ADC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000027CB661ADC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequentia

In [56]:
trainingName = "CIF_UE_MaxC10_C100R50_BS50" #Name of the current Training Scheme Used for Tensorboard and Saving Results
keys = ['loss','accuracy']
numberOfTraingRounds = 50
serverTestMetrics=[]*numberOfTraingRounds
logdir = "logs/" + trainingName + "/"
summary_writer = tf.summary.create_file_writer(logdir)
with summary_writer.as_default():
    for i in range(0,numberOfTraingRounds):
        serverFA.getClientActiveStatus()
        serverFA.getClientCIF()
        serverFA.getClientSelected()
        serverFA.getModelUpdateFromClients()
        metrics = serverFA.testServerModel()
        d_metrics = zip(keys,metrics)
        serverTestMetrics.append(metrics)
        serverFA.updateAllClients()
        for name , value in d_metrics:
            tf.summary.scalar(name, value, step=i)


--------------------------------------------
 Round NO: 0
Active Clients in Round :--:  [0, 5, 9, 10, 12, 15, 17, 18, 19, 20, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 34, 36, 38, 40, 44, 46, 49, 51, 55, 57, 59, 60, 61, 71, 73, 74, 77, 78, 79, 80, 81, 82, 84, 88, 91, 92, 94, 95, 97, 98]
CIF for Client:  0  is:  13.611928083896636
CIF for Client:  5  is:  15.141928083896637
CIF for Client:  9  is:  16.661928083896637
CIF for Client:  10  is:  14.071928083896637
CIF for Client:  12  is:  18.901928083896635
CIF for Client:  15  is:  16.31192808389664
CIF for Client:  17  is:  17.551928083896637
CIF for Client:  18  is:  13.231928083896637
CIF for Client:  19  is:  16.551928083896637
CIF for Client:  20  is:  17.081928083896635
CIF for Client:  23  is:  16.591928083896637
CIF for Client:  24  is:  14.151928083896637
CIF for Client:  25  is:  21.981928083896637
CIF for Client:  26  is:  19.961928083896638
CIF for Client:  27  is:  17.49192808389664
CIF for Client:  28  is:  18.571928083896637

CIF for Client:  3  is:  24.66424098134041
CIF for Client:  4  is:  18.50454996109009
CIF for Client:  5  is:  16.880270844697954
CIF for Client:  7  is:  18.156943485736846
CIF for Client:  8  is:  17.663284515142443
CIF for Client:  10  is:  15.518909772634506
CIF for Client:  12  is:  18.51726840019226
CIF for Client:  15  is:  15.24846710920334
CIF for Client:  18  is:  14.487452498674394
CIF for Client:  21  is:  19.13756397008896
CIF for Client:  22  is:  18.215383900403978
CIF for Client:  23  is:  18.83500836133957
CIF for Client:  25  is:  21.664470677375792
CIF for Client:  26  is:  20.006614805459975
CIF for Client:  28  is:  18.770027379989624
CIF for Client:  29  is:  19.74487684249878
CIF for Client:  30  is:  16.169355744123457
CIF for Client:  31  is:  19.323882578611375
CIF for Client:  32  is:  15.04237010538578
CIF for Client:  33  is:  18.437005537152288
CIF for Client:  34  is:  18.674297395944595
CIF for Client:  37  is:  18.63249853849411
CIF for Client:  40  is:

CIF for Client:  13  is:  19.57833413541317
CIF for Client:  14  is:  20.881311616301538
CIF for Client:  15  is:  18.046856949329374
CIF for Client:  17  is:  19.474994000792503
CIF for Client:  19  is:  15.542804731726646
CIF for Client:  22  is:  20.016653587818144
CIF for Client:  25  is:  20.585461852550505
CIF for Client:  27  is:  19.077462767362597
CIF for Client:  29  is:  20.55094622373581
CIF for Client:  30  is:  15.409422972798348
CIF for Client:  33  is:  19.410355318188667
CIF for Client:  35  is:  20.871570802927017
CIF for Client:  38  is:  17.36322412610054
CIF for Client:  41  is:  19.328487553596496
CIF for Client:  43  is:  17.893351459503172
CIF for Client:  44  is:  18.41124569892883
CIF for Client:  45  is:  18.7678559923172
CIF for Client:  46  is:  19.001578783392908
CIF for Client:  47  is:  20.418299868106843
CIF for Client:  48  is:  19.144955084323882
CIF for Client:  49  is:  16.652997061014176
CIF for Client:  50  is:  20.003755978345872
CIF for Client: 

CIF for Client:  39  is:  18.81997174859047
CIF for Client:  40  is:  16.399549028873444
CIF for Client:  47  is:  17.436402129530904
CIF for Client:  48  is:  18.122963494062425
CIF for Client:  50  is:  20.28233807325363
CIF for Client:  51  is:  19.41662440896034
CIF for Client:  52  is:  17.565311425328254
CIF for Client:  53  is:  17.05694671154022
CIF for Client:  54  is:  15.964471362829208
CIF for Client:  58  is:  20.662673416137697
CIF for Client:  62  is:  12.718952969908715
CIF for Client:  63  is:  17.944638991355895
CIF for Client:  67  is:  20.40113004028797
CIF for Client:  69  is:  21.43642611384392
CIF for Client:  70  is:  18.19879510819912
CIF for Client:  71  is:  19.118465623259546
CIF for Client:  75  is:  12.926915871500968
CIF for Client:  78  is:  14.171634072065354
CIF for Client:  80  is:  18.76286140561104
CIF for Client:  81  is:  15.855800419449807
CIF for Client:  82  is:  14.374462743997574
CIF for Client:  83  is:  11.254312416315079
CIF for Client:  8

CIF for Client:  31  is:  18.9827253562212
CIF for Client:  32  is:  16.503061202764513
CIF for Client:  35  is:  19.178128434419634
CIF for Client:  36  is:  18.99833947479725
CIF for Client:  37  is:  18.103824926614763
CIF for Client:  38  is:  15.782115894556046
CIF for Client:  39  is:  20.666766885519028
CIF for Client:  41  is:  20.74299296617508
CIF for Client:  44  is:  19.196357454657555
CIF for Client:  45  is:  18.498489285111425
CIF for Client:  48  is:  21.503679146170615
CIF for Client:  49  is:  17.228028705716135
CIF for Client:  50  is:  20.27852382719517
CIF for Client:  52  is:  16.973448044657708
CIF for Client:  55  is:  16.80644723594189
CIF for Client:  56  is:  16.792170697450636
CIF for Client:  58  is:  20.200115472078323
CIF for Client:  59  is:  13.151939593553543
CIF for Client:  60  is:  14.587374663352966
CIF for Client:  65  is:  16.09988874554634
CIF for Client:  66  is:  14.821662707328796
CIF for Client:  67  is:  20.759481617212295
CIF for Client:  

...

CIF for Client:  23  is:  16.871404254287484
CIF for Client:  25  is:  21.55474747210741
CIF for Client:  26  is:  18.314744900092485
CIF for Client:  27  is:  17.653892385533545
CIF for Client:  28  is:  17.143709765672682
CIF for Client:  29  is:  20.454410544335843
CIF for Client:  30  is:  15.692800732851028
CIF for Client:  31  is:  16.633589068651197
CIF for Client:  32  is:  13.613201957410201
CIF for Client:  33  is:  18.230360427051785
CIF for Client:  37  is:  18.514858396053313
CIF for Client:  38  is:  14.510855994820595
CIF for Client:  40  is:  13.8367450915277
CIF for Client:  43  is:  18.26996212951839
CIF for Client:  47  is:  18.5882385417819
CIF for Client:  50  is:  20.33613801538944
CIF for Client:  51  is:  19.180823556035758
CIF for Client:  52  is:  18.06471120715141
CIF for Client:  53  is:  15.38231787621975
CIF for Client:  55  is:  16.32831489443779
CIF for Client:  58  is:  19.63102822393179
CIF for Client:  65  is:  16.925691518187524
CIF for Client:  66  

CIF for Client:  1  is:  15.020630387663841
CIF for Client:  3  is:  19.219845016896723
CIF for Client:  7  is:  16.941920451372862
CIF for Client:  8  is:  17.54703852504492
CIF for Client:  12  is:  17.642539506554606
CIF for Client:  16  is:  16.741351665183902
CIF for Client:  18  is:  15.051980445981025
CIF for Client:  19  is:  15.883375833034515
CIF for Client:  20  is:  16.68928310200572
CIF for Client:  21  is:  20.362988462001084
CIF for Client:  22  is:  18.605535020232203
CIF for Client:  24  is:  11.763342362046242
CIF for Client:  25  is:  19.82753446623683
CIF for Client:  27  is:  17.20485420025885
CIF for Client:  28  is:  19.37848175942898
CIF for Client:  31  is:  19.69300094231963
CIF for Client:  33  is:  18.450726661980152
CIF for Client:  34  is:  17.050000752806664
CIF for Client:  35  is:  16.815421711206437
CIF for Client:  37  is:  17.161896492540837
CIF for Client:  41  is:  19.335383489727974
CIF for Client:  43  is:  17.117228823751212
CIF for Client:  44 

CIF for Client:  5  is:  15.270371287167073
CIF for Client:  6  is:  18.16295025408268
CIF for Client:  9  is:  15.641050468552857
CIF for Client:  10  is:  15.006299340277911
CIF for Client:  13  is:  19.901231799721717
CIF for Client:  16  is:  13.541545486599206
CIF for Client:  17  is:  17.775172293186188
CIF for Client:  19  is:  16.604387176036834
CIF for Client:  20  is:  16.060411197990177
CIF for Client:  21  is:  19.629385887086393
CIF for Client:  24  is:  11.182145521640777
CIF for Client:  25  is:  20.765817981921135
CIF for Client:  26  is:  18.13158185482025
CIF for Client:  29  is:  20.758042102232576
CIF for Client:  32  is:  13.815340245738625
CIF for Client:  35  is:  19.175027233958243
CIF for Client:  36  is:  17.882225018143654
CIF for Client:  40  is:  13.938514841943979
CIF for Client:  43  is:  18.2926752474159
CIF for Client:  44  is:  18.766074880957603
CIF for Client:  45  is:  17.964846296310427
CIF for Client:  46  is:  18.532022739648816
CIF for Client:  

32/32 [==============================] - 0s 1ms/step - loss: 0.6990 - accuracy: 0.8020
--------------------------------------------
 Round NO: 46
Active Clients in Round :--:  [1, 2, 3, 6, 10, 14, 17, 21, 26, 27, 30, 33, 35, 40, 41, 43, 46, 47, 50, 51, 52, 53, 55, 56, 57, 59, 62, 68, 70, 74, 76, 78, 80, 81, 82, 84, 85, 88, 93, 94, 97, 98]
CIF for Client:  1  is:  15.984358855485915
CIF for Client:  2  is:  16.163389638885853
CIF for Client:  3  is:  21.07321342781186
CIF for Client:  6  is:  17.31048456445336
CIF for Client:  10  is:  16.178989034015686
CIF for Client:  14  is:  18.158438554629683
CIF for Client:  17  is:  18.3723117017746
CIF for Client:  21  is:  18.574612954854963
CIF for Client:  26  is:  18.531311575546862
CIF for Client:  27  is:  18.643998661860824
CIF for Client:  30  is:  16.08257423788309
CIF for Client:  33  is:  18.707230266928672
CIF for Client:  35  is:  19.974163259863854
CIF for Client:  40  is:  16.95490031629801
CIF for Client:  41  is:  18.7976937246

CIF for Client:  3  is:  21.74460760846734
CIF for Client:  4  is:  16.494537408947945
CIF for Client:  5  is:  15.06497506827116
CIF for Client:  8  is:  18.061080552339554
CIF for Client:  9  is:  14.208511157780885
CIF for Client:  10  is:  15.840517798364163
CIF for Client:  12  is:  17.985628593564034
CIF for Client:  17  is:  18.572474159896373
CIF for Client:  18  is:  13.439420250654221
CIF for Client:  19  is:  17.851077429652214
CIF for Client:  23  is:  19.07677569806576
CIF for Client:  27  is:  19.638741662502287
CIF for Client:  28  is:  18.35284928679466
CIF for Client:  29  is:  19.894175416231157
CIF for Client:  30  is:  16.34501178354025
CIF for Client:  31  is:  17.584132543206216
CIF for Client:  34  is:  17.820513235032557
CIF for Client:  37  is:  18.262259806394574
CIF for Client:  42  is:  18.496628879755736
CIF for Client:  43  is:  19.98099480740726
CIF for Client:  48  is:  18.207586704790593
CIF for Client:  49  is:  16.393810617029665
CIF for Client:  51  

In [ ]:
plt.plot(serverTestMetrics)


In [ ]:
serverFA.getDataCost()

In [11]:
%tensorboard --logdir logs/ 

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 9596.

In [ ]:
testAcc=[]*numberOfTraingRounds
testLoss=[]*numberOfTraingRounds
for i in range(0,numberOfTraingRounds):
    testAcc.append(trainingMetrics[i][1])
    testLoss.append(trainingMetrics[i][0])
k = np.array(testLoss)

In [ ]:
%tensorboard --kill 30696

In [ ]:
# This Cell is used to get results for Server Only Training
#I.e. there are no clients and all the client dataset (Both Test and Train) are in server
# The server performs 10 epochs in one round and 50 round updates (self, no clients involved)
# The server train data set uses client IDs 1-100 same as federated learning scenario
# The server test dataset uses client IDs 80-100 same as federated learning scenario


trainingName = "ServerOnly_C1_100E10R50" #Name of the current Training Scheme Used for Tensorboard and Saving Results
#Clients Data used from IDs 1 to 100 as a part of server train and test data
keys = ['loss','accuracy']
numberOfTraingRounds = 50
serverTestMetrics=[]*numberOfTraingRounds
logdir = "logs/" + trainingName + "/"
summary_writer = tf.summary.create_file_writer(logdir)
with summary_writer.as_default():
    for i in range(0,numberOfTraingRounds):
        print('ROUNDNO:',i)
        serverFA.performServerTraining()
        metrics = serverFA.testServerModel()
        d_metrics = zip(keys,metrics)
        serverTestMetrics.append(metrics)
        serverFA.updateAllClients()
        for name , value in d_metrics:
            tf.summary.scalar(name, value, step=i)

In [9]:
# This cell is used to for pretraining demonstration
# The server model is pretrained for 50 epochs from the training dataset of Client IDs 2000-2500 before initial broadcast
# The server test dataset is kept same as before i.e. 80-500
# Since the dataset is in iterable format, the server pretraining is performed using 50 self round updates with 1 epochs each

#---- Pre-training of Server Model
for i in range(0,50):
    #print('ROUNDNO:',i)
    serverFA.performServerTraining()


# BroadCast the Model ( Do not broadcast the model before)
serverFA.initialBroadcast()

#--- Perform FA training
trainingName = "PreTrain2_UE_MaxC10_C100R50" #Name of the current Training Scheme Used for Tensorboard and Saving Results
keys = ['loss','accuracy']
numberOfTraingRounds = 50
serverTestMetrics=[]*numberOfTraingRounds
logdir = "logs/" + trainingName + "/"
summary_writer = tf.summary.create_file_writer(logdir)
with summary_writer.as_default():
    for i in range(0,numberOfTraingRounds):
        serverFA.getClientActiveStatus()
        serverFA.getClientCIF()
        serverFA.getClientSelected()
        serverFA.getModelUpdateFromClients()
        metrics = serverFA.testServerModel()
        d_metrics = zip(keys,metrics)
        serverTestMetrics.append(metrics)
        serverFA.updateAllClients()
        for name , value in d_metrics:
            tf.summary.scalar(name, value, step=i)


<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000026BE8C8E288>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000026BE8C8E288>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000026BE8C8E288>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000026BE8C8E288>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000026BE8C8E288>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000026BE8C8E288>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000026BE8C8E288>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000026BE8C8E288>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequentia

CIF for Client:  9  is:  13.071611657440663
CIF for Client:  10  is:  19.307250199615954
CIF for Client:  15  is:  16.655866373479366
CIF for Client:  16  is:  16.547020640075207
CIF for Client:  17  is:  12.1024979531765
CIF for Client:  18  is:  16.471421974897385
CIF for Client:  22  is:  18.796480627059935
CIF for Client:  23  is:  20.505327772200108
CIF for Client:  25  is:  12.480404996573926
CIF for Client:  28  is:  11.699947126209736
CIF for Client:  31  is:  19.109511826634407
CIF for Client:  33  is:  15.387672859430314
CIF for Client:  39  is:  17.221995826363564
CIF for Client:  40  is:  11.772256610691548
CIF for Client:  42  is:  14.922313461005688
CIF for Client:  44  is:  18.53044706314802
CIF for Client:  45  is:  15.970082181096076
CIF for Client:  46  is:  20.44484706580639
CIF for Client:  49  is:  18.894258270561696
CIF for Client:  51  is:  16.489815445840357
CIF for Client:  52  is:  17.74684448361397
CIF for Client:  54  is:  17.251078225970268
CIF for Client: 

CIF for Client:  2  is:  22.471538734436034
CIF for Client:  3  is:  14.464562348127366
CIF for Client:  5  is:  17.322243400812148
CIF for Client:  6  is:  18.25001832008362
CIF for Client:  9  is:  13.541529126167298
CIF for Client:  10  is:  22.412965064048766
CIF for Client:  11  is:  17.930554659366607
CIF for Client:  13  is:  17.940068566799162
CIF for Client:  16  is:  20.650224163532258
CIF for Client:  18  is:  16.87945401191711
CIF for Client:  21  is:  20.617119716405867
CIF for Client:  23  is:  19.12706925988197
CIF for Client:  24  is:  14.997074764966964
CIF for Client:  25  is:  13.183637342453004
CIF for Client:  26  is:  17.94539581477642
CIF for Client:  29  is:  16.109444485902785
CIF for Client:  30  is:  20.764278362989426
CIF for Client:  31  is:  22.016575436592102
CIF for Client:  32  is:  12.321460562944413
CIF for Client:  42  is:  14.960877632498741
CIF for Client:  44  is:  22.503812273740767
CIF for Client:  46  is:  23.003350834846497
CIF for Client:  47

CIF for Client:  16  is:  20.282641446590425
CIF for Client:  17  is:  14.143991907238961
CIF for Client:  18  is:  16.72546685278416
CIF for Client:  19  is:  17.389914268255232
CIF for Client:  20  is:  15.05438774585724
CIF for Client:  21  is:  19.09833334028721
CIF for Client:  22  is:  19.328510023355484
CIF for Client:  24  is:  12.385280593633652
CIF for Client:  25  is:  14.521980960965157
CIF for Client:  28  is:  14.94136364042759
CIF for Client:  32  is:  11.443878825306893
CIF for Client:  34  is:  18.64146885752678
CIF for Client:  36  is:  13.811494262814522
CIF for Client:  39  is:  17.961085862517358
CIF for Client:  42  is:  14.684000746011733
CIF for Client:  43  is:  14.49904183268547
CIF for Client:  44  is:  20.488021464347838
CIF for Client:  46  is:  22.524546487927438
CIF for Client:  48  is:  16.4638953858614
CIF for Client:  49  is:  17.35795205116272
CIF for Client:  50  is:  16.683794511556627
CIF for Client:  51  is:  17.124206770062447
CIF for Client:  52

CIF for Client:  11  is:  18.105156191289424
CIF for Client:  16  is:  19.678611460328103
CIF for Client:  17  is:  12.705185419917107
CIF for Client:  21  is:  20.738763719797134
CIF for Client:  22  is:  20.21574535429478
CIF for Client:  23  is:  18.773183579444883
CIF for Client:  24  is:  14.33208484172821
CIF for Client:  25  is:  13.744861208200454
CIF for Client:  30  is:  20.268170716166495
CIF for Client:  32  is:  13.83837826371193
CIF for Client:  33  is:  16.977455421090127
CIF for Client:  37  is:  16.30027118086815
CIF for Client:  38  is:  15.4268623816967
CIF for Client:  39  is:  19.380104516744613
CIF for Client:  40  is:  16.99285390436649
CIF for Client:  41  is:  19.89364548921585
CIF for Client:  42  is:  14.590197492837905
CIF for Client:  43  is:  14.444742437899112
CIF for Client:  44  is:  21.325060573816298
CIF for Client:  47  is:  13.660228359103202
CIF for Client:  48  is:  12.2996069252491
CIF for Client:  49  is:  18.46577175796032
CIF for Client:  50  

...

CIF for Client:  46  is:  20.768708313778042
CIF for Client:  47  is:  14.458182465992868
CIF for Client:  49  is:  18.54211331576109
CIF for Client:  50  is:  16.144217407107355
CIF for Client:  54  is:  18.53120931982994
CIF for Client:  55  is:  16.01348040530458
CIF for Client:  56  is:  20.900199602097274
CIF for Client:  57  is:  18.14121281120926
CIF for Client:  58  is:  15.038217428177596
CIF for Client:  61  is:  16.669363347887995
CIF for Client:  62  is:  15.742757170200347
CIF for Client:  63  is:  19.004234895408153
CIF for Client:  64  is:  22.18061201393604
CIF for Client:  65  is:  13.442767669558526
CIF for Client:  66  is:  19.500150834619998
CIF for Client:  67  is:  15.334986579418182
CIF for Client:  68  is:  19.952926994264125
CIF for Client:  70  is:  14.604571756273508
CIF for Client:  71  is:  16.61142511129379
CIF for Client:  73  is:  16.55516241632402
CIF for Client:  74  is:  12.653363505601883
CIF for Client:  75  is:  19.014473418593408
CIF for Client:  

CIF for Client:  35  is:  9.993862832784654
CIF for Client:  36  is:  12.060374957919121
CIF for Client:  41  is:  17.29149911224842
CIF for Client:  43  is:  15.531164738312363
CIF for Client:  45  is:  16.849452155828477
CIF for Client:  46  is:  20.306225441992282
CIF for Client:  50  is:  14.536116847991943
CIF for Client:  51  is:  14.636491962969304
CIF for Client:  52  is:  16.01474625170231
CIF for Client:  53  is:  17.694420456439257
CIF for Client:  55  is:  14.265526992175728
CIF for Client:  57  is:  20.628830775395034
CIF for Client:  58  is:  16.82856897622347
CIF for Client:  66  is:  17.9572191002965
CIF for Client:  67  is:  19.648456725478173
CIF for Client:  70  is:  14.87520975485444
CIF for Client:  71  is:  17.588321100473404
CIF for Client:  72  is:  19.172814927510917
CIF for Client:  73  is:  16.510456895530226
CIF for Client:  79  is:  15.04235508352518
CIF for Client:  81  is:  18.272137338519094
CIF for Client:  82  is:  15.897139652594923
CIF for Client:  8

CIF for Client:  24  is:  12.877713231146336
CIF for Client:  39  is:  17.045958002805712
CIF for Client:  43  is:  15.976733306720853
CIF for Client:  45  is:  15.475552004277706
CIF for Client:  46  is:  20.59761059820652
CIF for Client:  47  is:  14.160122565329075
CIF for Client:  48  is:  13.728944271951914
CIF for Client:  50  is:  15.789539803862572
CIF for Client:  51  is:  17.247727686464785
CIF for Client:  52  is:  17.57230974137783
CIF for Client:  57  is:  17.644653315842152
CIF for Client:  60  is:  16.39982167005539
CIF for Client:  63  is:  17.394936313629152
CIF for Client:  64  is:  23.469629531055688
CIF for Client:  65  is:  13.33768841266632
CIF for Client:  66  is:  16.815305646955967
CIF for Client:  67  is:  19.112479756474496
CIF for Client:  68  is:  19.019176255762577
CIF for Client:  69  is:  20.610858644805848
CIF for Client:  71  is:  16.089035353660584
CIF for Client:  77  is:  19.624396335184574
CIF for Client:  81  is:  18.22960363075137
CIF for Client:

CIF for Client:  17  is:  15.034217315763236
CIF for Client:  18  is:  18.551519200205803
CIF for Client:  19  is:  17.688850898444652
CIF for Client:  20  is:  15.492919272184372
CIF for Client:  21  is:  19.57109018832445
CIF for Client:  24  is:  13.53384731799364
CIF for Client:  26  is:  16.07716081563383
CIF for Client:  27  is:  9.100747266203399
CIF for Client:  37  is:  15.437586083859205
CIF for Client:  38  is:  15.820375159829855
CIF for Client:  39  is:  18.512079776227473
CIF for Client:  41  is:  17.521304878890515
CIF for Client:  42  is:  13.154770414195955
CIF for Client:  49  is:  19.7079565474391
CIF for Client:  51  is:  15.644854910671711
CIF for Client:  53  is:  18.021614134162665
CIF for Client:  57  is:  18.468559134565293
CIF for Client:  61  is:  17.83496894054115
CIF for Client:  65  is:  13.407713794112205
CIF for Client:  67  is:  17.265803639143705
CIF for Client:  69  is:  22.555541580058634
CIF for Client:  73  is:  16.19330928295851
CIF for Client:  7

CIF for Client:  9  is:  13.620370470434427
CIF for Client:  15  is:  16.9473664149642
CIF for Client:  20  is:  13.641590382307768
CIF for Client:  22  is:  20.184976714849473
CIF for Client:  24  is:  13.431426895260811
CIF for Client:  28  is:  14.628502842783927
CIF for Client:  29  is:  17.183019745498896
CIF for Client:  31  is:  20.780179355815054
CIF for Client:  32  is:  11.652918665558099
CIF for Client:  33  is:  16.894142899513245
CIF for Client:  34  is:  17.07721519306302
CIF for Client:  35  is:  12.090473015606403
CIF for Client:  36  is:  11.885517863184214
CIF for Client:  38  is:  15.469482214450837
CIF for Client:  43  is:  15.156714989170432
CIF for Client:  44  is:  22.773931373655795
CIF for Client:  46  is:  20.700338558554648
CIF for Client:  48  is:  15.437579908072948
CIF for Client:  49  is:  19.290068581774833
CIF for Client:  50  is:  17.18204339414835
CIF for Client:  51  is:  17.271579118967054
CIF for Client:  52  is:  15.370942093729973
CIF for Client:

In [35]:
# This cell is used for performing federated averaging with completely random clients NO CIF or Client Active functions are used

trainingName = "COM_RAND_MaxC10_C100R50" #Name of the current Training Scheme Used for Tensorboard and Saving Results
keys = ['loss','accuracy']
numberOfTraingRounds = 50
serverTestMetrics=[]*numberOfTraingRounds
logdir = "logs/" + trainingName + "/"
summary_writer = tf.summary.create_file_writer(logdir)
with summary_writer.as_default():
    for i in range(0,numberOfTraingRounds):
        serverFA.getRandomClients()
        serverFA.getModelUpdateFromClients()
        metrics = serverFA.testServerModel()
        d_metrics = zip(keys,metrics)
        serverTestMetrics.append(metrics)
        serverFA.updateAllClients()
        for name , value in d_metrics:
            tf.summary.scalar(name, value, step=i)


Active Clients:  []
CIF of Active Clients:  []
Selected Clients:  [44 45 18 55 82 31 59 45 43 20]
32/32 [==============================] - 0s 1ms/step - loss: 2.2358 - accuracy: 0.3260
Active Clients:  []
CIF of Active Clients:  []
Selected Clients:  [28 25 96 36  5 72 30 92 58 63]
32/32 [==============================] - 0s 2ms/step - loss: 2.1087 - accuracy: 0.2780
Active Clients:  []
CIF of Active Clients:  []
Selected Clients:  [23 33 32 23 63 22 29 23 52 82]
32/32 [==============================] - 0s 1ms/step - loss: 2.0613 - accuracy: 0.3340
Active Clients:  []
CIF of Active Clients:  []
Selected Clients:  [16 20 96 65 67 29 19 81 99 32]
32/32 [==============================] - 0s 1ms/step - loss: 1.9185 - accuracy: 0.5090
Active Clients:  []
CIF of Active Clients:  []
Selected Clients:  [44 65 75 57 92 13 79 68 29 81]
32/32 [==============================] - 0s 1ms/step - loss: 1.8423 - accuracy: 0.5200
Active Clients:  []
CIF of Active Clients:  []
Selected Clients:  [77  1 87

32/32 [==============================] - 0s 1ms/step - loss: 0.8583 - accuracy: 0.7910
Active Clients:  []
CIF of Active Clients:  []
Selected Clients:  [23 51  5 34 26 82 86 57 59  7]
32/32 [==============================] - 0s 1ms/step - loss: 0.8770 - accuracy: 0.7740
Active Clients:  []
CIF of Active Clients:  []
Selected Clients:  [ 8  5 33 24 36 93 78 40 43  8]
32/32 [==============================] - 0s 1ms/step - loss: 0.8643 - accuracy: 0.7920
Active Clients:  []
CIF of Active Clients:  []
Selected Clients:  [37 24 55 48 84 47 75  9 60 64]
32/32 [==============================] - 0s 1ms/step - loss: 0.8531 - accuracy: 0.7870
Active Clients:  []
CIF of Active Clients:  []
Selected Clients:  [15 32 12 11 92 69 27 17 70 44]
32/32 [==============================] - 0s 1ms/step - loss: 0.8633 - accuracy: 0.7690
Active Clients:  []
CIF of Active Clients:  []
Selected Clients:  [99 23 97 13 49 23 31 39 78 68]
32/32 [==============================] - 0s 1ms/step - loss: 0.8582 - accur

In [ ]:
np.savetxt("loss_history.txt", np.array(testLoss))

In [ ]:
meanSINR = int(np.random.normal(15, 2.5, 1))
print(meanSINR)

In [ ]:
np.round((np.random.normal(meanSINR, 1, 1)[0]), 2)

In [ ]:
raise SystemExit("Stop right there!")


In [ ]:
# calculate the information for a dice roll
from math import log2
# probability of the event
p = 1.0 / 6.0
# calculate information for event
h = -log2(p)
# print the result
print('p(x)=%.3f, information: %.3f bits' % (p, h))

In [ ]:
h

In [ ]:
probs = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

In [ ]:
from matplotlib import pyplot
info = [-log2(p) for p in probs]
# plot probability vs information
pyplot.plot(probs, info, marker='.')
pyplot.title('Probability vs Information')
pyplot.xlabel('Probability')
pyplot.ylabel('Information')
pyplot.show()